# Segmenting and Clustering Neighborhoods in Toronto 

### So let's import some important libraries

In [1]:
!pip install BeautifulSoup4
!pip install lxml

In [2]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen
print('Everything ok so far!')

Everything ok so far!


### Then, scrape the wikipedia page and turn it into a DF!

In [3]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')

df = pd.read_html(str(table))


### Now clean the data


In [4]:
canadata=pd.DataFrame(df[0])
index = canadata[canadata['Borough'] == 'Not assigned'].index
canadata.drop(index, inplace=True)

print("Cleaned!")

Cleaned!



### Organize


In [5]:
canadata = canadata.sort_values(by=['Postal Code','Borough'])
canadata.reset_index(drop=True, inplace=True)
print("Organized!")

Organized!



### Print the shape and head!



In [6]:
print(canadata.shape)
canadata

(103, 3)


,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."



### And now, let's get the geospatial data!


In [7]:
!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [8]:
url = "http://cocl.us/Geospatial_data"
geodf = pd.read_csv(url)
geodf.columns = ['GeoPostCode','Latitude', 'Longitude']
geodf


,GeoPostCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437



### I need to glue all together!


In [9]:
canadataplus=pd.concat([canadata, geodf], axis=1)
canadataplus

,Postal Code,Borough,Neighbourhood,GeoPostCode,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
...,...,...,...,...,...,...
98,M9N,York,Weston,M9N,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,M9P,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",M9R,43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",M9V,43.739416,-79.588437



### Oooops, two Postal code! Let's drop!


In [10]:
canadataplus.drop('GeoPostCode', axis=1, inplace=True)
canadataplus

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


## Time to explore!

#### First of all, let me see it in a map

In [11]:
import folium

#### Let's get the Canada coordinates for this map

In [12]:
address = 'Canada'

### Got this in internet to make things easier

latitude = 43.651070
longitude = -79.347015
print('The geograpical coordinate of Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Canada are 43.65107, -79.347015.


In [13]:
map_canada = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, borough, neighborhood in zip(canadataplus['Latitude'], canadataplus['Longitude'], canadataplus['Borough'], canadataplus['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada

#### Now, it's a good time to slice and look closer!

##### First of all, I want to see the unique values of each column and choose a place to look closer

In [14]:
canadataplus['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

In [15]:
canadataplus['Neighbourhood'].unique()

array(['Malvern, Rouge', 'Rouge Hill, Port Union, Highland Creek',
       'Guildwood, Morningside, West Hill', 'Woburn', 'Cedarbrae',
       'Scarborough Village',
       'Kennedy Park, Ionview, East Birchmount Park',
       'Golden Mile, Clairlea, Oakridge',
       'Cliffside, Cliffcrest, Scarborough Village West',
       'Birch Cliff, Cliffside West',
       'Dorset Park, Wexford Heights, Scarborough Town Centre',
       'Wexford, Maryvale', 'Agincourt',
       "Clarks Corners, Tam O'Shanter, Sullivan",
       "Milliken, Agincourt North, Steeles East, L'Amoreaux East",
       "Steeles West, L'Amoreaux West", 'Upper Rouge',
       'Hillcrest Village', 'Fairview, Henry Farm, Oriole',
       'Bayview Village', 'York Mills, Silver Hills',
       'Willowdale, Newtonbrook', 'Willowdale, Willowdale East',
       'York Mills West', 'Willowdale, Willowdale West', 'Parkwoods',
       'Don Mills', 'Bathurst Manor, Wilson Heights, Downsview North',
       'Northwood Park, York University', 'Down

#### I'm interested in see Downtown. I like the place were the University is, so I'm going to fix my analysis in the neighborhood of 'University of Toronto'!

In [16]:
toronto_data = canadataplus[canadataplus['Neighbourhood'] == 'University of Toronto, Harbord'].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049


I will gonna try Nominatim. I need to confess: I didn't try it at first to make the first map!

In [17]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [18]:
address = "University of Toronto, Toronto, ON"
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of University of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of University of Toronto are 43.663461999999996, -79.39775965337452.


In [19]:
map_dtoronto = folium.Map(location=[latitude, longitude], zoom_start=16)


for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dtoronto)  
    
map_dtoronto

#### That's right! Ok, now I can define my access to Foursquare!

In [20]:
CLIENT_ID = 'UOS5SO2GPBYR5KGDTN1DHIJ3E4AOMUBRNIHZFRLNTHUYL0K4' 
CLIENT_SECRET = 'FMZH1TDQJ5OFH51OZ5WBHNWMGAPODHT2GHDARR4LKT1MVDFF'
VERSION = '20180605'

print('My credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentials:
CLIENT_ID: UOS5SO2GPBYR5KGDTN1DHIJ3E4AOMUBRNIHZFRLNTHUYL0K4
CLIENT_SECRET:FMZH1TDQJ5OFH51OZ5WBHNWMGAPODHT2GHDARR4LKT1MVDFF


### Cool!
##### I'm goin to make some Foursquare magic here and...

#### Create the URL

In [21]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=UOS5SO2GPBYR5KGDTN1DHIJ3E4AOMUBRNIHZFRLNTHUYL0K4&client_secret=FMZH1TDQJ5OFH51OZ5WBHNWMGAPODHT2GHDARR4LKT1MVDFF&v=20180605&ll=43.663461999999996,-79.39775965337452&radius=500&limit=100'

#### Get the JSON
## Important - I decided not to print the results, because it is a long, long list!

In [22]:
results = requests.get(url).json()
print("We got results - but I'm not gonna print it because it is long!")

We got results - but I'm not gonna print it because it is long!


#### Run the function to normalize, clean and turn it into a beautiful pandas dataframe!

In [23]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Philosopher's Walk,Park,43.666894,-79.395597
1,Hart House Theatre,Theater,43.663571,-79.394616
2,Yasu,Japanese Restaurant,43.662837,-79.403217
3,Athletic Centre,College Gym,43.662487,-79.400657
4,Queen's Park,Park,43.663946,-79.392180


#### How many venues?

In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

31 venues were returned by Foursquare.


## My idea here is explore the places as if I'm an international student: I want to know were to go between classes!

I need to mark where the University is

In [27]:
university_address = "27 King's College Cir, Toronto, ON M5S, Canada"

geolocator = Nominatim(user_agent="foursquare_agent")
ulocation = geolocator.geocode(address)
ulatitude = location.latitude
ulongitude = location.longitude
print(ulatitude, ulongitude)

43.663461999999996 -79.39775965337452


In [28]:
venues_map = folium.Map(location=[ulatitude, ulongitude], zoom_start=16) 

# add a red circle marker to represent the University
folium.features.CircleMarker(
    [ulatitude, ulongitude],
    radius=10,
    color='red',
    popup='University of Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the venues as blue circle markers
for lat, lng, label in zip(nearby_venues.lat, nearby_venues.lng, nearby_venues.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
 
    
# display map
venues_map

## Time to cluster!

#### I'm gonna take the venues of all places of dowtwon now

In [29]:
downtown_data = canadataplus[canadataplus['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Defining the venues

In [31]:
downtown_venues = getNearbyVenues(names=downtown_data['Neighbourhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government


In [32]:
downtown_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"St. James Town, Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner
...,...,...,...,...,...,...,...
1247,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Convocation Hall,43.660828,-79.395245,College Auditorium
1248,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,SUDS,43.659880,-79.394712,Bar
1249,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Tim Hortons,43.658906,-79.388696,Coffee Shop
1250,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,The Green Beet Cafe,43.662096,-79.394153,Vegetarian / Vegan Restaurant


### Let's count the results we got

In [33]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,65,65,65,65,65,65
Christie,18,18,18,18,18,18
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


### Applying ine hot encode

In [34]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

#drop neighborhood
downtown_onehot.drop('Neighborhood', axis=1, inplace=True)

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Time to group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [35]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000000,0.000000,0.00,...,0.00,0.00,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.0625,0.0625,0.0625,0.0625,0.125,0.125,0.000000,0.000000,0.00,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Central Bay Street,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000000,0.000000,0.00,...,0.00,0.00,0.00,0.015385,0.000000,0.000000,0.015385,0.000000,0.000000,0.015385
3,Christie,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000000,0.000000,0.00,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.012987,0.000000,0.00,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025974
5,"Commerce Court, Victoria Hotel",0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.040000,0.000000,0.00,...,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
6,"First Canadian Place, Underground city",0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.030000,0.000000,0.00,...,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
7,"Garden District, Ryerson",0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000000,0.000000,0.00,...,0.01,0.00,0.00,0.000000,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000000,0.000000,0.05,...,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000000,0.000000,0.00,...,0.00,0.00,0.00,0.043478,0.000000,0.043478,0.014493,0.000000,0.014493,0.000000


#### And print with the top venues - yes, just like NY!

In [36]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1        Bakery  0.03
2  Cocktail Bar  0.03
3      Beer Bar  0.03
4           Pub  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0      Airport Service  0.12
1     Airport Terminal  0.12
2              Airport  0.06
3      Harbor / Marina  0.06
4  Rental Car Location  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1      Sandwich Place  0.06
2                Café  0.05
3  Italian Restaurant  0.05
4        Burger Joint  0.03


----Christie----
           venue  freq
0  Grocery Store  0.22
1           Café  0.17
2           Park  0.11
3      Nightclub  0.06
4    Candy Store  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.09
1  Japanese Restaurant  0.06
2     Sushi Restaurant  0.05
3              Gay Bar  0.05
4           Restaurant  0.0

#### Applying the function

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### And create the dataframe!

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Cheese Shop,Farmers Market,Seafood Restaurant,Bakery,Restaurant,Pub,Beer Bar,Cocktail Bar
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport,Bar,Plane,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Coffee Shop
2,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Modern European Restaurant,New American Restaurant
3,Christie,Grocery Store,Café,Park,Bank,Baby Store,Italian Restaurant,Athletics & Sports,Restaurant,Diner,Nightclub
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Yoga Studio


## Time to cluster!

In [39]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [40]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 3, 0, 2, 4, 4, 4, 4, 4, 4], dtype=int32)

#### Again - dataframe it!

In [41]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

downtown_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Yoga Studio,Deli / Bodega,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,4,Café,Coffee Shop,Restaurant,Pizza Place,Italian Restaurant,Bakery,Pub,Chinese Restaurant,Convenience Store,Park
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,4,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Yoga Studio
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4,Coffee Shop,Bakery,Pub,Café,Park,Breakfast Spot,Theater,Yoga Studio,Mexican Restaurant,Shoe Store
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Ramen Restaurant,Bookstore,Pizza Place


### In the NY Notebook they created a map, so let's do it!

In [43]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighbourhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## And, the finale: examinate clusters

Cluster 1

In [49]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,0,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Modern European Restaurant,New American Restaurant
18,Downtown Toronto,0,Coffee Shop,Yoga Studio,Persian Restaurant,Distribution Center,Sandwich Place,Diner,Japanese Restaurant,Italian Restaurant,Beer Bar,Smoothie Shop


Cluster 2

In [50]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Deli / Bodega,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


Cluster 3

In [51]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,3,Airport Service,Airport Terminal,Airport,Bar,Plane,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Coffee Shop


Cluster 4

In [52]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,4,Café,Coffee Shop,Restaurant,Pizza Place,Italian Restaurant,Bakery,Pub,Chinese Restaurant,Convenience Store,Park
2,Downtown Toronto,4,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Yoga Studio
3,Downtown Toronto,4,Coffee Shop,Bakery,Pub,Café,Park,Breakfast Spot,Theater,Yoga Studio,Mexican Restaurant,Shoe Store
4,Downtown Toronto,4,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Ramen Restaurant,Bookstore,Pizza Place
5,Downtown Toronto,4,Coffee Shop,Café,Cocktail Bar,American Restaurant,Farmers Market,Hotel,Gym,Gastropub,Clothing Store,Italian Restaurant
6,Downtown Toronto,4,Coffee Shop,Café,Cheese Shop,Farmers Market,Seafood Restaurant,Bakery,Restaurant,Pub,Beer Bar,Cocktail Bar
8,Downtown Toronto,4,Coffee Shop,Café,Clothing Store,Restaurant,Gym,Bar,Hotel,Thai Restaurant,Salad Place,Concert Hall
9,Downtown Toronto,4,Coffee Shop,Aquarium,Restaurant,Hotel,Café,Scenic Lookout,Fried Chicken Joint,Brewery,History Museum,Plaza
10,Downtown Toronto,4,Coffee Shop,Hotel,Café,Restaurant,Italian Restaurant,Gastropub,Seafood Restaurant,Salad Place,American Restaurant,Japanese Restaurant
11,Downtown Toronto,4,Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Gastropub,Seafood Restaurant,Japanese Restaurant,Deli / Bodega
